In [1]:
from all_functions import *
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv
import torch
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.torch import DeepAREstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from mango import scheduler, Tuner
import random
import torch
torch.set_float32_matmul_precision('medium')


def get_train_test_deepar(train_norm, test_norm):
    concat_norm = pd.concat([train_norm, test_norm])
    dataset_norm = PandasDataset(concat_norm, target="value")
    # test_ds = PandasDataset(test_norm, target="value")
    training_data, test_gen = split(dataset_norm, offset=-12)
    test_data = test_gen.generate_instances(prediction_length=12, windows=1)
    return training_data, test_data

def set_seed(seed_value):
    np.random.seed(seed_value)
    random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)

# Definir a semente desejada
seed_value = 42
set_seed(seed_value)

In [9]:
global horizon
window = 12
horizon = 12
derivado = "querosenedeaviacao"
estado = "RR"
series = read_series(f'../datasets/venda/mensal/uf/{derivado}/mensal_{estado.lower()}_{derivado}.csv')
train_test_splits = []
min_train_size = 36

aux_series = series
while len(aux_series) > horizon + min_train_size:
    train, test = aux_series[:-horizon], aux_series[-horizon:]
    train_test_splits.append((train, test))
    aux_series = train
#{'context_length': 12, 'num_layers': 2, 'batch_size': 32, 'max_epochs': 30, 'num_batches_per_epoch': 20}

train_test_splits[19]

(timestamp
 1990-01    553.8920
 1990-02    675.2730
 1990-03     37.5320
 1990-04    354.1935
 1990-05    670.8550
              ...   
 2003-10    645.6520
 2003-11      0.0000
 2003-12    303.5550
 2004-01    320.7440
 2004-02    308.7130
 Freq: M, Name: m3, Length: 170, dtype: float64,
 timestamp
 2004-03     39.100
 2004-04    311.866
 2004-05    606.889
 2004-06    312.031
 2004-07    646.565
 2004-08    759.830
 2004-09    405.617
 2004-10    490.394
 2004-11    388.312
 2004-12    669.795
 2005-01    212.143
 2005-02    495.181
 Freq: M, Name: m3, dtype: float64)